# Récupération jeu de données

L'objectif de ce notebook

Pour la documentation api Spotify



*   https://spotifypy.readthedocs.io/en/latest/api.html
*   https://github.com/spotipy-dev/spotipy/blob/master/examples/search.py
*   https://pypi.org/project/spotipy/








## Installation de l'api SPOTIFY

In [1]:
 # Installation des packages pour Spotify
#pip install spotipy 

## Import des librairies

In [2]:
from function.utils import tqdm, glob, json, os
from function.utils import requests, BeautifulSoup #TODO : ligne a supprimer.

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Récupération des tokens

/!\ Attention /!\  
Pour pouvoir récuperer les access tokens de spotify il est impératif d'avoir un compte free ou premium.  
De plus avoir créer son APP.  
Par la suite, vous obtiendrez les client_id et client_id_secret permettant l'accès à l'API et le bon fonctionnement du code.

In [3]:
from function.function import get_token

PATH_TOKEN = "../data/token/*"
PATH_JSON = "../data/json_file/"
PATH_TXT = "../data/txt/"

liste_txt_file = glob.glob(PATH_TOKEN)

GENIUS_ACCESS_TOKEN = None
SPOTIFY_client_id = None
SPOTIFY_client_secret = None

GENIUS_ACCESS_TOKEN, SPOTIFY_client_id, SPOTIFY_client_secret = get_token(liste_txt_file)


In [4]:
from function.dictionnary import get_dictionnary

dictionnaire_song = get_dictionnary(PATH_JSON, "dictionnaire_song2.json")
liste_artiste = get_dictionnary(PATH_JSON, "dictionnaire_artiste.json")
data_artiste_titre = get_dictionnary(PATH_JSON, "data_artiste_titre.json")

In [5]:
#identifiant spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_client_id,
                                                           client_secret=SPOTIFY_client_secret))

data_artiste_titre = list(data_artiste_titre)

if len(data_artiste_titre) == 0:
    for nom_artiste in tqdm.tqdm(liste_artiste):
            results = sp.search(q=nom_artiste, limit=50) # requete de la recherche
            if 'tracks' in results and 'items' in results['tracks']:
                Info_titre = [(idx, nom_artiste, data['name']) for idx, data in enumerate(results['tracks']['items']) ]
            data_artiste_titre.extend(Info_titre)
    with open(PATH_JSON+"data_artiste_titre.txt", "w") as json_file:
        json.dump(data_artiste_titre, json_file, indent=4)
    

In [6]:
# from function.function import get_lyrics_from_genius

# song_title = "She Will"
# #song_title = "Rich Baby Daddy (feat. Sexyy Red & SZA)"

# artist_name = "drake"

# test, type_artiste = get_lyrics_from_genius(song_title, artist_name, GENIUS_ACCESS_TOKEN)

# print(type_artiste, test)

In [ ]:
from function.dictionnary import update_dictionary

headers = {"User-Agent": "Mozilla/5.0"}

if os.path.exists("checkpoint.txt"):
    with open("checkpoint.txt", "r") as f:
        start_index = int(f.read().strip())
else:
    start_index = 0

save_every = 1000
    
#TODO : Dans le scrapping il récupère aussi des livres     
for idx in tqdm.tqdm(range(start_index, len(data_artiste_titre))):
    artist_name, song_title = data_artiste_titre[idx][1:]
    try:
        
        if idx % save_every == 0:
            with open("checkpoint.txt", "w") as w:
                w.write(str(idx))

        # Si l'artiste est dans le dictionnaire et que le titre est dans le dictionnaire de l'artist
        if artist_name in dictionnaire_song and song_title in dictionnaire_song[artist_name].keys():

            # Si les paroles sont None tu mets à jour
            if dictionnaire_song[artist_name][song_title]["lyrics_primaire"] == None or dictionnaire_song[artist_name][song_title]["lyrics_primaire"] == "":
                dictionnaire_song = update_dictionary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)
            # Sinon tu passes
            continue
        else:
            dictionnaire_song = update_dictionary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)

    except Exception as e:
        print(e)
        #print(f"erreur à l'index {idx} : {e}")

        with open("checkpoint.txt", "w") as w:
            w.write(str(idx))
        break


  1%|▏         | 2444/171074 [4:01:43<44363:06:37, 947.09s/it]

Erreur pendant la requête Genius : HTTPSConnectionPool(host='api.genius.com', port=443): Max retries exceeded with url: /search?q=When+The+Lights+Go+Down (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F6DB9D2DD0>: Failed to resolve 'api.genius.com' ([Errno 11001] getaddrinfo failed)"))


  1%|▏         | 2514/171074 [4:07:47<120:25:42,  2.57s/it]   

In [ ]:
with open(PATH_JSON+"dictionnaire_song2.json", "w") as json_file:
    json.dump(dictionnaire_song, json_file, indent=4)

In [ ]:
# def scrapping_find_lyrics_on_lyricsfind(lyrics_url, headers):
#     response = requests.get(lyrics_url, headers=headers)
#     soup = BeautifulSoup(response.text, "html.parser")
#     print(soup)
#     1/0
#     lyrics_div = soup.find_all("div", {"class": "MuiBox-root css-0"})
#     liste_parole = [div.get_text(separator="\n").strip() for div in lyrics_div]
#     return liste_parole

In [ ]:
# import json #TODO : a supprimer

# headers = {"User-Agent": "Mozilla/5.0"}

# with open(PATH_JSON+"dictionnaire_song2.json", "r") as json_file:
#     dictionnaire_song = json.load(json_file)

# for artiste, dictionnaire_titre in dictionnaire_song.items():
#     for titre, lyrics in dictionnaire_titre.items():
#         if lyrics["lyrics"] is None:
#             print(artiste, slug(titre), lyrics["lyrics"])
#             #liste_parole = scrapping_find_lyrics_on_lyricsfind(f"https://lyrics.lyricfind.com/lyrics/{artiste} {titre}", headers)
#         #print(lyrics.values())
#         #print(" ".join(lyrics.values()))
#         #1/0